In [1]:

import pandas as pd
import numpy as np
import pickle

# Leer embeddings de habilidades
skill_embeddings = np.load("data/skill_embeddings.npy")

# Leer habilidades únicas
with open("data/unique_skills.pkl", "rb") as f:
    unique_skills = pickle.load(f)

# Leer DataFrame con clústeres de habilidades
skill_cluster_df = pd.read_csv("data/skill_cluster_df.csv")

# Diccionario: habilidad -> clúster
with open("data/skill_to_cluster.pkl", "rb") as f:
    skill_to_cluster = pickle.load(f)

# Leer DataFrame combinado de ofertas
combined_df = pd.read_csv("data/combined_df.csv")




C:\Users\maria\AppData\Local\Temp\ipykernel_19392\3554055149.py:20: DtypeWarning: Columns (0,1,2,3,5,6,7,8,9,10,11,12,17,21,22,23,24,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv("data/combined_df.csv")


In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar el modelo de embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

# Vectorizar la palabra/habilidad introducida
query = "oroject management"
query_vector = model.encode([query])


c:\Users\maria\anaconda3\envs\tfm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# skill_embeddings ya está cargado y corresponde a unique_skills
similaridades = cosine_similarity(query_vector, skill_embeddings)[0]

# Buscar la habilidad más parecida
indice_top = np.argmax(similaridades)
habilidad_mas_cercana = unique_skills[indice_top]
similitud_top = similaridades[indice_top]

print(f"🔍 '{query}' se parece más a '{habilidad_mas_cercana}' (similitud={similitud_top:.2f})")


🔍 'oroject management' se parece más a 'administration and management' (similitud=0.32)


In [4]:
cluster_asignado = skill_to_cluster.get(habilidad_mas_cercana)
print(f"🧠 Se asigna al clúster de habilidades: {cluster_asignado}")


🧠 Se asigna al clúster de habilidades: 9


In [5]:
combined_df["job_title"] = combined_df["job title"].fillna(combined_df["title"])


In [6]:
def map_skill_clusters_robusta(skill_list):
    if not isinstance(skill_list, str):
        return []
    
    # Separar correctamente por coma y limpiar espacios
    skills = [s.strip().lower() for s in skill_list.split(",") if s.strip()]
    
    # Mapear a clústeres
    return [skill_to_cluster.get(s) for s in skills if skill_to_cluster.get(s) is not None]


In [7]:
combined_df["skill_clusters"] = combined_df["skills_extraidas"].apply(map_skill_clusters_robusta)


In [8]:
# Verificar que ahora sí haya ofertas con el clúster 15
ofertas_cl15 = combined_df[combined_df["skill_clusters"].apply(lambda cl: isinstance(cl, list) and 15 in cl)]
print(f"✅ Ofertas donde skill_clusters incluye el clúster 15: {len(ofertas_cl15)}")


✅ Ofertas donde skill_clusters incluye el clúster 15: 496


In [9]:
# Convertir todo a string y eliminar valores puramente numéricos
def limpiar_job_title(title):
    try:
        float(title)
        return np.nan  # Si es numérico, lo quitamos
    except:
        return str(title).strip()

combined_df["job_title"] = combined_df["job_title"].apply(limpiar_job_title)



In [10]:
from sklearn.preprocessing import LabelEncoder

# Guardar la columna original antes de transformarla
combined_df["job_title_original"] = combined_df["job_title"]

# Asegurarse de que todos los valores sean string
combined_df["job_title"] = combined_df["job_title"].astype(str).fillna("unknown")

# Aplicar label encoding
le = LabelEncoder()
combined_df["job_title_encoded"] = le.fit_transform(combined_df["job_title"])

# Crear mapeo inverso
title_mapping = dict(zip(combined_df["job_title_encoded"], combined_df["job_title_original"]))


In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
combined_df["job_title_encoded"] = le.fit_transform(combined_df["job_title"])


# Crear diccionario inverso
title_mapping = dict(zip(combined_df["job_title_encoded"], combined_df["job_title_original"]))


In [12]:
from collections import Counter
import numpy as np

def analizar_puesto2(titulo_puesto, combined_df, skill_cluster_df, top_n=10, modelo_embeddings=None):
    titulo_puesto = titulo_puesto.lower()

    # 1. Filtrar ofertas cuyo título coincida parcial o totalmente
    ofertas_filtradas = combined_df[
        combined_df["job_title"].str.lower().str.contains(titulo_puesto, na=False)
        | combined_df["title"].str.lower().str.contains(titulo_puesto, na=False)
    ]
    
    if ofertas_filtradas.empty:
        print("❌ No se encontraron ofertas con ese título.")
        return {}

    print(f"🔍 Se encontraron {len(ofertas_filtradas)} ofertas para el puesto '{titulo_puesto}'")

    # 2. Obtener las habilidades más comunes
    skills_series = ofertas_filtradas["skills_extraidas"].dropna().str.lower().str.split(", ")
    skills_flat = [s.strip() for sublist in skills_series for s in sublist if s.strip()]
    top_skills = Counter(skills_flat).most_common(top_n)

    # 3. Identificar clústeres de habilidades asociados
    clusters_encontrados = set()
    for lista in ofertas_filtradas["skill_clusters"].dropna():
        if isinstance(lista, list):
            clusters_encontrados.update(lista)
        elif isinstance(lista, str) and lista.startswith("["):
            try:
                clusters_encontrados.update(eval(lista))
            except:
                continue

    # 4. Buscar otras ofertas con esos mismos clústeres de habilidades
    similares_habilidades = combined_df[combined_df["skill_clusters"].apply(
        lambda sc: any(c in clusters_encontrados for c in sc) if isinstance(sc, list) else False
    )]

    # 5. 🔥 Buscar el clúster de ofertas más común en las ofertas encontradas
    if "job_cluster" in combined_df.columns and not ofertas_filtradas["job_cluster"].isnull().all():
        cluster_oferta_dominante = ofertas_filtradas["job_cluster"].dropna().mode().iloc[0]
        similares_ofertas = combined_df[combined_df["job_cluster"] == cluster_oferta_dominante]
    else:
        cluster_oferta_dominante = None
        similares_ofertas = None

    # 6. Analizar puestos similares en ambos enfoques usando similitud semántica
    puestos_similares = []
    puestos_en_cluster = []

    # --- Por similitud semántica (habilidades)
    if modelo_embeddings is not None:
        emb_target = modelo_embeddings.encode([titulo_puesto])[0]

        # Del conjunto por habilidades
        puestos_unicos_hab = similares_habilidades["job_title"].dropna().unique()
        similitudes_hab = []
        for puesto in puestos_unicos_hab:
            emb = modelo_embeddings.encode([puesto])[0]
            sim = cosine_similarity([emb_target], [emb])[0][0]
            similitudes_hab.append((puesto, sim))

        puestos_similares = sorted(similitudes_hab, key=lambda x: x[1], reverse=True)[:top_n]

        similitudes_cluster = []  # 🔧 inicializar antes del if

        if similares_ofertas is not None:
            puestos_unicos_cluster = similares_ofertas["job_title"].dropna().unique()
            for puesto in puestos_unicos_cluster:
                emb = modelo_embeddings.encode([puesto])[0]
                sim = cosine_similarity([emb_target], [emb])[0][0]
                similitudes_cluster.append((puesto, sim))

        puestos_en_cluster = sorted(similitudes_cluster, key=lambda x: x[1], reverse=True)[:top_n]


        puestos_en_cluster = sorted(similitudes_cluster, key=lambda x: x[1], reverse=True)[:top_n]
    else:
        # Fallback: usar frecuencia si no hay modelo
        puestos_similares = similares_habilidades["job_title"].value_counts().head(top_n).items()
        puestos_en_cluster = (
            similares_ofertas["job_title"].value_counts().head(top_n).items()
            if similares_ofertas is not None else []
        )


    # 6B. Obtener salario medio por cada uno de los puestos similares semánticamente
    salario_por_puesto_similar = {}
    for puesto, _ in puestos_similares:
        subset_salario = similares_habilidades[similares_habilidades["job_title"] == puesto]["avg_salary"].dropna()
        if not subset_salario.empty:
            salario_por_puesto_similar[puesto] = round(subset_salario.mean(), 2)

    # 7. Salario medio general
    salario_medio_habilidades = similares_habilidades["avg_salary"].dropna().mean()
    salario_medio_cluster = similares_ofertas["avg_salary"].dropna().mean() if similares_ofertas is not None else None

    # 8. Salario medio por habilidad más común
    habilidades_comunes = [h[0] for h in top_skills]
    salario_por_habilidad = {}
    for habilidad in habilidades_comunes:
        ofertas_con_habilidad = combined_df[
            combined_df["skills_extraidas"].str.contains(habilidad, na=False, case=False)
        ]
        salario_prom = ofertas_con_habilidad["avg_salary"].dropna().mean()
        if not np.isnan(salario_prom):
            salario_por_habilidad[habilidad] = round(salario_prom, 2)

    # 9. Salario medio por puesto similar
    salarios_por_puesto = similares_habilidades.dropna(subset=["job_title", "avg_salary"]).groupby("job_title")["avg_salary"].mean()
    puestos_disponibles = [p for p, _ in puestos_similares if p in salarios_por_puesto.index]
    salarios_top_puestos = salarios_por_puesto.loc[puestos_disponibles].round(2).to_dict()

    # 10. Salario medio por puesto en el clúster de ofertas
    if similares_ofertas is not None:
        salarios_cluster_por_puesto = similares_ofertas.dropna(subset=["job_title", "avg_salary"]).groupby("job_title")["avg_salary"].mean()
        puestos_cluster = [p for p in puestos_en_cluster.index if p in salarios_cluster_por_puesto.index]
        salarios_cluster_top = salarios_cluster_por_puesto.loc[puestos_cluster].round(2).to_dict()
    else:
        salarios_cluster_top = {}

    return {
        "ofertas_encontradas": len(ofertas_filtradas),
        "habilidades_mas_comunes": top_skills,
        "salario_medio_por_habilidad_comun": salario_por_habilidad,
        "clusters_habilidad_hallados": list(clusters_encontrados),
        "puestos_similares_por_habilidad": dict(puestos_similares),
        "salario_medio_por_puesto_similar_habilidad": salarios_top_puestos,
        "salario_medio_por_puesto_similar_semantico": salario_por_puesto_similar,
        "salario_medio_ofertas_similares_por_habilidad": round(salario_medio_habilidades, 2) if not np.isnan(salario_medio_habilidades) else None,
        "cluster_oferta_dominante": cluster_oferta_dominante,
        "puestos_similares_por_clúster_de_oferta": dict(puestos_en_cluster),
        "salario_medio_por_puesto_clúster_oferta": salarios_cluster_top,
        "salario_medio_ofertas_clúster": round(salario_medio_cluster, 2) if salario_medio_cluster is not None and not np.isnan(salario_medio_cluster) else None
    }


In [13]:
analizar_puesto2("data scientist", combined_df, skill_cluster_df, modelo_embeddings=model)


🔍 Se encontraron 7096 ofertas para el puesto 'data scientist'


{'ofertas_encontradas': 7096,
 'habilidades_mas_comunes': [('design', 3577),
  ('programming', 3487),
  ('transportation', 1378),
  ('science', 141),
  ('mathematics', 94),
  ('physics', 53),
  ('monitoring', 6),
  ('writing', 6),
  ('coordination', 6),
  ('critical thinking', 3)],
 'salario_medio_por_habilidad_comun': {'design': np.float64(83181.16),
  'programming': np.float64(83723.09),
  'transportation': np.float64(82526.79),
  'science': np.float64(110867.05),
  'mathematics': np.float64(197967.22),
  'physics': np.float64(197957.96),
  'monitoring': np.float64(84308.45),
  'writing': np.float64(83102.27),
  'coordination': np.float64(82979.74),
  'critical thinking': np.float64(82911.08)},
 'clusters_habilidad_hallados': [33, 2, 3, 1, 7, 12, 13, 25, 26],
 'puestos_similares_por_habilidad': {'Data Scientist': np.float32(1.0000001),
  'data scientist': np.float32(1.0000001),
  'head of data science': np.float32(0.8490081),
  'data scientist ad tech': np.float32(0.8336302),
  'data

In [14]:
analizar_puesto2("data engineer", combined_df, skill_cluster_df, modelo_embeddings=model)

🔍 Se encontraron 10610 ofertas para el puesto 'data engineer'


{'ofertas_encontradas': 10610,
 'habilidades_mas_comunes': [('design', 3621),
  ('transportation', 2124),
  ('science', 100),
  ('programming', 70),
  ('troubleshooting', 25),
  ('writing', 12),
  ('monitoring', 7),
  ('time management', 6),
  ('mathematics', 4),
  ('coordination', 3)],
 'salario_medio_por_habilidad_comun': {'design': np.float64(83181.16),
  'transportation': np.float64(82526.79),
  'science': np.float64(110867.05),
  'programming': np.float64(83723.09),
  'troubleshooting': np.float64(82618.34),
  'writing': np.float64(83102.27),
  'monitoring': np.float64(84308.45),
  'time management': np.float64(82834.49),
  'mathematics': np.float64(197967.22),
  'coordination': np.float64(82979.74)},
 'clusters_habilidad_hallados': [33, 2, 3, 1, 4, 7, 12, 26, 29],
 'puestos_similares_por_habilidad': {'Data Engineer': np.float32(1.0),
  'data engineer': np.float32(1.0),
  'data engineer iv': np.float32(0.89425325),
  'staff data engineer': np.float32(0.8453232),
  'data analytics 

In [15]:
analizar_puesto2("project manager", combined_df, skill_cluster_df, modelo_embeddings=model)

🔍 Se encontraron 15305 ofertas para el puesto 'project manager'


{'ofertas_encontradas': 15305,
 'habilidades_mas_comunes': [('coordination', 3658),
  ('negotiation', 3517),
  ('transportation', 2892),
  ('design', 556),
  ('monitoring', 278),
  ('science', 185),
  ('time management', 162),
  ('administrative', 131),
  ('mechanical', 108),
  ('installation', 106)],
 'salario_medio_por_habilidad_comun': {'coordination': np.float64(82979.74),
  'negotiation': np.float64(82813.62),
  'transportation': np.float64(82526.79),
  'design': np.float64(83181.16),
  'monitoring': np.float64(84308.45),
  'science': np.float64(110867.05),
  'time management': np.float64(82834.49),
  'administrative': np.float64(82478.2),
  'mechanical': np.float64(82918.58),
  'installation': np.float64(83873.16)},
 'clusters_habilidad_hallados': [0,
  1,
  2,
  3,
  4,
  5,
  7,
  8,
  9,
  10,
  12,
  13,
  18,
  19,
  21,
  25,
  26,
  28,
  29,
  30,
  31,
  33],
 'puestos_similares_por_habilidad': {'Project Manager': np.float32(0.9999999),
  'project manager': np.float32(0.

In [16]:
analizar_puesto2("account manager", combined_df, skill_cluster_df, modelo_embeddings=model)

🔍 Se encontraron 17892 ofertas para el puesto 'account manager'


{'ofertas_encontradas': 17892,
 'habilidades_mas_comunes': [('negotiation', 10468),
  ('transportation', 3550),
  ('writing', 92),
  ('sales and marketing', 58),
  ('time management', 55),
  ('troubleshooting', 49),
  ('design', 42),
  ('coordination', 37),
  ('monitoring', 32),
  ('science', 29)],
 'salario_medio_por_habilidad_comun': {'negotiation': np.float64(82813.62),
  'transportation': np.float64(82526.79),
  'writing': np.float64(83102.27),
  'sales and marketing': np.float64(108796.34),
  'time management': np.float64(82834.49),
  'troubleshooting': np.float64(82618.34),
  'design': np.float64(83181.16),
  'coordination': np.float64(82979.74),
  'monitoring': np.float64(84308.45),
  'science': np.float64(110867.05)},
 'clusters_habilidad_hallados': [1,
  2,
  3,
  4,
  5,
  7,
  8,
  9,
  10,
  12,
  13,
  15,
  18,
  19,
  25,
  26,
  28,
  29,
  30,
  31,
  33],
 'puestos_similares_por_habilidad': {'Account Manager': np.float32(1.0),
  'account manager': np.float32(1.0),
  '

In [17]:
analizar_puesto2("account manager", combined_df, skill_cluster_df, modelo_embeddings=model)

🔍 Se encontraron 17892 ofertas para el puesto 'account manager'


{'ofertas_encontradas': 17892,
 'habilidades_mas_comunes': [('negotiation', 10468),
  ('transportation', 3550),
  ('writing', 92),
  ('sales and marketing', 58),
  ('time management', 55),
  ('troubleshooting', 49),
  ('design', 42),
  ('coordination', 37),
  ('monitoring', 32),
  ('science', 29)],
 'salario_medio_por_habilidad_comun': {'negotiation': np.float64(82813.62),
  'transportation': np.float64(82526.79),
  'writing': np.float64(83102.27),
  'sales and marketing': np.float64(108796.34),
  'time management': np.float64(82834.49),
  'troubleshooting': np.float64(82618.34),
  'design': np.float64(83181.16),
  'coordination': np.float64(82979.74),
  'monitoring': np.float64(84308.45),
  'science': np.float64(110867.05)},
 'clusters_habilidad_hallados': [1,
  2,
  3,
  4,
  5,
  7,
  8,
  9,
  10,
  12,
  13,
  15,
  18,
  19,
  25,
  26,
  28,
  29,
  30,
  31,
  33],
 'puestos_similares_por_habilidad': {'Account Manager': np.float32(1.0),
  'account manager': np.float32(1.0),
  '

In [18]:
analizar_puesto2("customer service", combined_df, skill_cluster_df, modelo_embeddings=model)

🔍 Se encontraron 17974 ofertas para el puesto 'customer service'


{'ofertas_encontradas': 17974,
 'habilidades_mas_comunes': [('transportation', 3534),
  ('troubleshooting', 3465),
  ('speaking', 168),
  ('writing', 143),
  ('monitoring', 21),
  ('mechanical', 19),
  ('negotiation', 19),
  ('science', 16),
  ('administrative', 16),
  ('coordination', 15)],
 'salario_medio_por_habilidad_comun': {'transportation': np.float64(82526.79),
  'troubleshooting': np.float64(82618.34),
  'speaking': np.float64(89362.84),
  'writing': np.float64(83102.27),
  'monitoring': np.float64(84308.45),
  'mechanical': np.float64(82918.58),
  'negotiation': np.float64(82813.62),
  'science': np.float64(110867.05),
  'administrative': np.float64(82478.2),
  'coordination': np.float64(82979.74)},
 'clusters_habilidad_hallados': [1,
  2,
  3,
  4,
  5,
  7,
  8,
  9,
  10,
  12,
  13,
  17,
  18,
  25,
  26,
  28,
  29,
  31],
 'puestos_similares_por_habilidad': {'Customer Service Manager': np.float32(0.82712436),
  'medical customer service': np.float32(0.8237847),
  'cust

In [19]:
analizar_puesto2("product manager", combined_df, skill_cluster_df, modelo_embeddings=model)

🔍 Se encontraron 7259 ofertas para el puesto 'product manager'


{'ofertas_encontradas': 7259,
 'habilidades_mas_comunes': [('writing', 3463),
  ('transportation', 1421),
  ('design', 218),
  ('science', 182),
  ('monitoring', 79),
  ('sales and marketing', 22),
  ('mathematics', 18),
  ('speaking', 16),
  ('coordination', 11),
  ('negotiation', 11)],
 'salario_medio_por_habilidad_comun': {'writing': np.float64(83102.27),
  'transportation': np.float64(82526.79),
  'design': np.float64(83181.16),
  'science': np.float64(110867.05),
  'monitoring': np.float64(84308.45),
  'sales and marketing': np.float64(108796.34),
  'mathematics': np.float64(197967.22),
  'speaking': np.float64(89362.84),
  'coordination': np.float64(82979.74),
  'negotiation': np.float64(82813.62)},
 'clusters_habilidad_hallados': [1, 2, 3, 33, 4, 7, 8, 12, 26, 28, 29, 31],
 'puestos_similares_por_habilidad': {'Product Manager': np.float32(0.99999994),
  'product manager': np.float32(0.99999994),
  'technical product manager': np.float32(0.850435),
  'associate product manager': 

In [20]:
def sugerir_por_palabras_clave_v2(
    queries,
    vector_model,
    skill_embeddings,
    unique_skills,
    skill_to_cluster,
    skill_cluster_df,
    combined_df,
    top_n=10
):
    """
    Devuelve recomendaciones a partir de un conjunto de habilidades (queries),
    adjuntando salario medio por habilidad y por puesto, siguiendo el estilo
    de 'analizar_puesto2'.
    """

    # ---------------------------
    # 1) Vector de consulta
    # ---------------------------
    vectores_query = [vector_model.encode(q.strip().lower()) for q in queries]
    query_vector = np.mean(vectores_query, axis=0)

    # ---------------------------
    # 2) Habilidad más cercana y su clúster
    # ---------------------------
    similitudes = np.dot(skill_embeddings, query_vector) / (
        np.linalg.norm(skill_embeddings, axis=1) * np.linalg.norm(query_vector)
    )
    idx_max = int(np.argmax(similitudes))
    habilidad_cercana = unique_skills[idx_max]
    similitud = float(similitudes[idx_max])

    cluster_asignado = skill_to_cluster.get(habilidad_cercana)
    if cluster_asignado is None:
        print("❌ La habilidad más cercana no tiene clúster asignado.")
        return {}

    # Habilidades que pertenecen al clúster detectado
    habilidades_en_cluster = (
        skill_cluster_df.loc[skill_cluster_df["cluster"] == cluster_asignado, "skill"]
        .astype(str)
        .str.lower()
        .tolist()
    )

    # ---------------------------
    # 3) Ofertas relacionadas (contienen ese clúster de habilidades)
    # ---------------------------
    ofertas_relacionadas = combined_df[combined_df["skill_clusters"].apply(
        lambda cl: isinstance(cl, list) and cluster_asignado in cl
    )].copy()

    # 3A) Limpieza de títulos (válidos)
    ofertas_relacionadas["job_title"] = ofertas_relacionadas["job_title"].astype(str)
    ofertas_relacionadas = ofertas_relacionadas[
        ofertas_relacionadas["job_title"].str.contains(r"[a-zA-Z]", na=False)
    ].copy()

    # ---------------------------
    # 4) Puestos más frecuentes (top_n)
    # ---------------------------
    puestos_relevantes = ofertas_relacionadas["job_title"].value_counts().head(top_n)

    # ---------------------------
    # 5) Salarios agregados (estilo analizar_puesto2)
    # ---------------------------
    # 5A) Salario medio global del subconjunto (fallback)
    salario_medio_subconjunto = ofertas_relacionadas["avg_salary"].dropna().mean()
    salario_medio_subconjunto = (
        round(float(salario_medio_subconjunto), 2)
        if salario_medio_subconjunto is not None and not np.isnan(salario_medio_subconjunto)
        else None
    )

    # 5B) Salario medio por PUESTO dentro del subconjunto (como en los pasos 6B y 9)
    salarios_por_puesto = (
        ofertas_relacionadas
        .dropna(subset=["job_title", "avg_salary"])
        .groupby("job_title")["avg_salary"]
        .mean()
        .round(2)
    )  # Series index=job_title, value=avg_salary

    # 5C) Salario medio por HABILIDAD individual (como en el paso 8)
    #     Usamos la columna de skills extraídas (lista tipo "a, b, c") si está disponible.
    salario_por_habilidad = {}
    if "skills_extraidas" in combined_df.columns:
        # Explode-like manual: buscamos ocurrencias por 'contains' (case-insensitive)
        for habilidad in habilidades_en_cluster:
            try:
                ofertas_con_habilidad = combined_df[
                    combined_df["skills_extraidas"].str.contains(habilidad, na=False, case=False)
                ]
                prom = ofertas_con_habilidad["avg_salary"].dropna().mean()
                if prom is not None and not np.isnan(prom):
                    salario_por_habilidad[habilidad] = round(float(prom), 2)
            except Exception:
                # Si hay formatos raros en skills_extraidas, ignoramos esa habilidad
                continue

    # ---------------------------
    # 6) Armar salidas “con salario” para la tabla
    # ---------------------------
    # 6A) Habilidades en el clúster (con salario). Fallback: salario_medio_subconjunto si falta.
    habilidades_en_cluster_con_salario = []
    for s in habilidades_en_cluster:
        if s in salario_por_habilidad:
            habilidades_en_cluster_con_salario.append((s, salario_por_habilidad[s]))
        else:
            habilidades_en_cluster_con_salario.append((s, salario_medio_subconjunto))

    # 6B) Puestos recomendados (con salario). Fallback: salario_medio_subconjunto si ese título no tiene media.
    puestos_recomendados_con_salario = []
    for titulo in puestos_relevantes.index.tolist():
        if titulo in salarios_por_puesto.index:
            puestos_recomendados_con_salario.append((titulo, float(salarios_por_puesto.loc[titulo])))
        else:
            puestos_recomendados_con_salario.append((titulo, salario_medio_subconjunto))

    # ---------------------------
    # 7) Clúster de ofertas dominante (opcional, como en analizar_puesto2)
    # ---------------------------
    if "job_cluster" in ofertas_relacionadas.columns and not ofertas_relacionadas["job_cluster"].isnull().all():
        cluster_oferta_dominante = ofertas_relacionadas["job_cluster"].dropna().mode().iloc[0]
        ofertas_clúster_oferta = combined_df[combined_df["job_cluster"] == cluster_oferta_dominante].copy()

        ofertas_clúster_oferta["job_title"] = ofertas_clúster_oferta["job_title"].astype(str)
        ofertas_clúster_oferta = ofertas_clúster_oferta[
            ofertas_clúster_oferta["job_title"].str.contains(r"[a-zA-Z]", na=False)
        ].copy()

        puestos_en_clúster_oferta = ofertas_clúster_oferta["job_title"].value_counts().head(top_n)
        salario_medio_clúster_oferta = ofertas_clúster_oferta["avg_salary"].dropna().mean()
        salario_medio_clúster_oferta = (
            round(float(salario_medio_clúster_oferta), 2)
            if salario_medio_clúster_oferta is not None and not np.isnan(salario_medio_clúster_oferta)
            else None
        )
    else:
        cluster_oferta_dominante = None
        puestos_en_clúster_oferta = pd.Series(dtype=int)
        salario_medio_clúster_oferta = None

    # ---------------------------
    # 8) Salida compatible con tus tablas
    # ---------------------------
    return {
        "query": queries,
        "habilidad_mas_cercana": habilidad_cercana,
        "similitud": similitud,
        "cluster_habilidad": cluster_asignado,

        # Habilidades
        "habilidades_en_cluster": habilidades_en_cluster,  # lista plana (por si la usas)
        "habilidades_en_cluster_con_salario": habilidades_en_cluster_con_salario,  # [(habilidad, salario)]

        # Puestos (frecuencia y con salario)
        "puestos_recomendados_por_habilidad": puestos_relevantes,  # pd.Series title->count
        "puestos_recomendados_con_salario": puestos_recomendados_con_salario,  # [(titulo, salario)]

        # Salarios agregados del subconjunto
        "salario_medio_en_ofertas_con_habilidad": salario_medio_subconjunto,

        # Clúster de ofertas dominante y su info
        "cluster_oferta_dominante": cluster_oferta_dominante,
        "puestos_recomendados_por_clúster_oferta": puestos_en_clúster_oferta,  # pd.Series
        "salario_medio_en_clúster_oferta": salario_medio_clúster_oferta
    }

In [21]:
# 🔎 Ejemplo 1: estadísticas y machine learning
res_stats = sugerir_por_palabras_clave_v2(
    ["statistics", "machine learning", "data analysis"],
    model,
    skill_embeddings,
    unique_skills,
    skill_to_cluster,
    skill_cluster_df,
    combined_df,
    top_n=10
)

# 🔎 Ejemplo 2: habilidades sociales
res_social = sugerir_por_palabras_clave_v2(
    ["communication", "teamwork", "leadership"],
    model,
    skill_embeddings,
    unique_skills,
    skill_to_cluster,
    skill_cluster_df,
    combined_df,
    top_n=10
)

# 🔎 Ejemplo 3: habilidades técnicas de programación
res_programming = sugerir_por_palabras_clave_v2(
    ["Python", "TensorFlow", "SQL"],
    model,
    skill_embeddings,
    unique_skills,
    skill_to_cluster,
    skill_cluster_df,
    combined_df,
    top_n=10
)


In [22]:
def formatear_lista_con_salario(lista):
    """
    Convierte [(item, salario), ...] en "item1 (salario), item2 (salario)"
    Si salario es None → pone '—'.
    """
    return ", ".join([f"{item} ({salario if salario is not None else '—'})" for item, salario in lista])

# Ejemplo de uso:
print("Habilidades en clúster:", formatear_lista_con_salario(res_stats["habilidades_en_cluster_con_salario"]))
print("Puestos recomendados:", formatear_lista_con_salario(res_stats["puestos_recomendados_con_salario"]))
print("Salario medio en ofertas con la habilidad:", res_stats["salario_medio_en_ofertas_con_habilidad"])


Habilidades en clúster: systems analysis (82891.21), systems evaluation (75000.0)
Puestos recomendados: Systems Analyst (82461.7), it project manager application development (145600.0), information technology manager ii (170547.5), electrical engineer (82890.1), software engineering analyst ii (107200.0), global project manager customer success strategy (113350.0), sr full stack net developer contract (183560.0), operations research analyst senior (133900.0), general engineer (82890.1), net developer production support (105000.0)
Salario medio en ofertas con la habilidad: 82890.1


In [23]:
for nombre, resultado in {
    "Statistics / ML / Data Analysis": res_stats,
    "Communication / Teamwork / Leadership": res_social,
    "Python / TensorFlow / SQL": res_programming
}.items():
    print(f"\n Resultados para: {nombre}")
    print("Habilidad más cercana:", resultado["habilidad_mas_cercana"])
    print("Clúster de habilidades:", resultado["cluster_habilidad"])
    print("Habilidades en clúster:", formatear_lista_con_salario(resultado["habilidades_en_cluster_con_salario"]))
    print("Puestos recomendados:", formatear_lista_con_salario(resultado["puestos_recomendados_con_salario"]))
    print("Salario medio en ofertas con la habilidad:", resultado["salario_medio_en_ofertas_con_habilidad"])



 Resultados para: Statistics / ML / Data Analysis
Habilidad más cercana: systems analysis
Clúster de habilidades: 21
Habilidades en clúster: systems analysis (82891.21), systems evaluation (75000.0)
Puestos recomendados: Systems Analyst (82461.7), it project manager application development (145600.0), information technology manager ii (170547.5), electrical engineer (82890.1), software engineering analyst ii (107200.0), global project manager customer success strategy (113350.0), sr full stack net developer contract (183560.0), operations research analyst senior (133900.0), general engineer (82890.1), net developer production support (105000.0)
Salario medio en ofertas con la habilidad: 82890.1

 Resultados para: Communication / Teamwork / Leadership
Habilidad más cercana: coordination
Clúster de habilidades: 7
Habilidades en clúster: coordination (82979.74)
Puestos recomendados: Executive Assistant (82488.69), Event Planner (82504.99), Event Coordinator (82525.48), Administrative Ass